In [1]:
# importing libraries
import pandas as pd 
import time
import mysql.connector

In [2]:
# Defining a function to read all datasets
def read_csv(file_name):
    return pd.read_csv(f"C:\\Python_Files\\Crime_data\\CRM_DT_SEG\\{file_name}.csv")

crime_df = read_csv("crime_tbl")
location_df = read_csv("location_tbl")
premise_df = read_csv("premise_tbl")
status_df = read_csv("status_tbl")
victim_df = read_csv("victim_tbl")
incidence_df = read_csv("incidence_reporting_tbl")

In [3]:
# Establishing connection with mysql workbench
my_db = mysql.connector.connect(
    host ="localhost",
    user = "root",
    password ="Ch!D0N!3SQL",
    port = 3306
)

In [4]:
# Creating the LAPD Crime Database
mycursor = my_db.cursor()

mycursor.execute("DROP DATABASE IF EXISTS `lapd_crm_rec`")
my_db.commit()

mycursor = my_db.cursor()

mycursor.execute("CREATE DATABASE IF NOT EXISTS `lapd_crm_rec`")
my_db.commit()

mycursor = my_db.cursor()
mycursor.execute("USE `lapd_crm_rec`")
my_db.commit()


# Establishing a connection with lapd_crm_rec
my_db = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = "Ch!D0N!3SQL",
    database = "lapd_crm_rec",
    port = 3306
)

In [5]:
# Creating tables in lapd_crm_rec
def cr8_schema(sql):
    mycursor = my_db.cursor()
    mycursor.execute(sql)
    my_db.commit()
    return True

crm_cr8_sql = '''CREATE TABLE IF NOT EXISTS crime(`crime_code` INT PRIMARY KEY NOT NULL,
crime_description VARCHAR(100) NULL)
'''
cr8_schema(crm_cr8_sql)

lctn_cr8_sql ='''CREATE TABLE IF NOT EXISTS location(`area_code` INT PRIMARY KEY NOT NULL,
`area_name` CHAR(80) NULL)
'''
cr8_schema(lctn_cr8_sql)

status_cr8_sql = '''CREATE TABLE IF NOT EXISTS status(`status_code` CHAR(5) PRIMARY KEY NOT NULL, 
`status_description` CHAR(20) NULL)
'''
cr8_schema(status_cr8_sql)

premis_cr8_sql = '''CREATE TABLE IF NOT EXISTS premise(`premise_code` INT PRIMARY KEY NOT NULL,
`premise_description` VARCHAR(80) NULL)
'''
cr8_schema(premis_cr8_sql)

incd_cr8_sql = '''CREATE TABLE IF NOT EXISTS incidence_reporting(`division_record_no` INT PRIMARY KEY NOT NULL,
`date_reported` DATETIME NULL,
`timestamp` DATETIME NULL,
`reporting_district_no` INT NULL,
`part_1-2` INT NULL,
`status_code` CHAR(5) NULL,
`crime_code` INT NULL,
`premise_code` INT NULL,
`area_code` INT NULL,
`location` VARCHAR(80) NULL,
`longitude` DOUBLE NULL,
`latitude` DOUBLE NULL,
FOREIGN KEY (`crime_code`) REFERENCES crime(`crime_code`),
FOREIGN KEY (`area_code`) REFERENCES location(`area_code`),
FOREIGN KEY (`premise_code`) REFERENCES premise(`premise_code`),
FOREIGN KEY (`status_code`) REFERENCES status(`status_code`)
)
'''
cr8_schema(incd_cr8_sql)

vict_cr8_sql = '''CREATE TABLE IF NOT EXISTS victim(`division_record_no` INT NULL,
`victim_age` INT NULL,
`victim_sex` CHAR(5) NULL,
`victim_descent` CHAR(5) NULL,
FOREIGN KEY (`division_record_no`) REFERENCES incidence_reporting(`division_record_no`))
'''
cr8_schema(vict_cr8_sql)

True

In [6]:
# SQL insert queries
crm_insert_sql = '''INSERT INTO crime(`crime_code`, `crime_description`) VALUES(%s, %s)'''

lctn_insert_sql = '''INSERT INTO location(`area_code`, `area_name`) VALUES(%s, %s)'''

status_insert_sql ='''INSERT INTO status(`status_code`, `status_description`) VALUES(%s, %s)'''

premis_insert_sql = '''INSERT INTO premise(`premise_code`, `premise_description`) VALUES(%s, %s)'''

incd_insert_sql = '''
    INSERT INTO incidence_reporting(`division_record_no`, `date_reported`, `timestamp`, 
    `reporting_district_no`, `part_1-2`, `status_code`, `crime_code`, `premise_code`, `area_code`, 
    `location`, `longitude`, `latitude`) VALUES(%s, %s, %s , %s, %s, %s, %s, %s, %s, %s, %s, %s)'''

vict_insert_sql = '''INSERT INTO victim(`division_record_no`, `victim_age`, `victim_sex`, `victim_descent`)
VALUES(%s, %s, %s, %s)
'''

In [7]:
%%time
# Defining function to insert records into tables
def insert_rec(dataframe):
    mycursor = my_db.cursor()
    if dataframe.equals(crime_df):
        try:
            records = [(row["Crm_Cd"], row["Crm_Cd_Desc"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(crm_insert_sql, records)
            my_db.commit()
            print(f"{len(records)} records inserted into crime table!")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
    elif dataframe.equals(location_df):
        try:
            records = [(row["AREA_Cd"], row["AREA_NAME"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(lctn_insert_sql,records)
            my_db.commit()
            print(f"{len(records)} records inserted into location table!")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
    elif dataframe.equals(premise_df):
        try:
            records = [(row["Premis_Cd"], row["Premis_Desc"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(premis_insert_sql, records)
            my_db.commit()
            print(f"{len(records)} records inserted into premise table!")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
    elif dataframe.equals(status_df):
        try:
            records = [(row["Status_Cd"], row["Status_Desc"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(status_insert_sql, records)
            my_db.commit()
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
    elif dataframe.equals(victim_df):
        try:
            records = [(row["DIV_REC_NO"], row["Vict_Age"], row["Vict_Sex"], row["Vict_Descent"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(vict_insert_sql, records)
            my_db.commit()
            print(f"{len(records)} records inserted into victim table!")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
    elif dataframe.equals(incidence_df):
        try:
            records = [(row["DIV_REC_NO"], row["Date_Rptd"], row["TIMESTAMP"], row["Rpt_Dist_No"], row["Part_1-2"], row["Status_Cd"], 
                        row["Crm_Cd"], row["Premis_Cd"], row["AREA_Cd"], row["LOCATION"], row["LAT"], row["LON"]) for _, row in dataframe.iterrows()]
            mycursor.executemany(incd_insert_sql, records)
            my_db.commit()
            print(f"{len(records)} records inserted into incidence reporting table!")
        except mysql.connector.Error as err:
            print(f"Error: {err}")
            my_db.rollback()
            
insert_rec(crime_df)
insert_rec(location_df)
insert_rec(premise_df)
insert_rec(status_df)
insert_rec(incidence_df)
insert_rec(victim_df)

136 records inserted into crime table!
21 records inserted into location table!
304 records inserted into premise table!
530681 records inserted into incidence reporting table!
530681 records inserted into victim table!
Wall time: 5min 26s
